In [1]:
import yt.wrapper as yt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind
from collections import Counter
import os

from tqdm.auto import tqdm

import json


%matplotlib inline

## Основные числа

Всего заданий **73**

Согласовано среди всех размечающих данное задание:
* Разметка кайфуль: 34 (минимум 3 человека размечали и это все примеры из "не было ничего крутого")
* Разметка провалов: 30 (минимум 3 человека, случаи сбалансированы)

Хотя бы 3 согласовано
* Разметка кайфуль: 50 (есть и гуды)
* Разметка провалов: 45 

In [2]:
yt.config["proxy"]["url"] = 'hahn.yt.yandex.net'
client = yt.YtClient(proxy='hahn', token=os.environ.get('YT_TOKEN'))

In [3]:
toloka_results = pd.read_csv("cool.tsv", sep="\t")
ilnur_table = read_result = pd.DataFrame(
    client.read_table("//home/voice/eliseevmax/tasks/EXPERIMENTS-99465/general_conversation_3_len/small_smart_speakers-quasar/session_len_join_freq", raw=False)
)

In [4]:
toloka_results["session_id"] = toloka_results["INPUT:input"].apply(
    lambda x: json.loads(x)["session_id"]
)

In [5]:
toloka_results["is_was_cool"] = 1*(toloka_results["OUTPUT:is_good"] == "yes")
toloka_results["is_was_not_cool"] = 1*(toloka_results["OUTPUT:is_good"] == "no")


toloka_results["is_was_bad"] = 1*(toloka_results["OUTPUT:is_bad"] == "yes")
toloka_results["is_was_not_bad"] = 1*(toloka_results["OUTPUT:is_bad"] == "no")

In [6]:
toloka_results

,INPUT:input,OUTPUT:is_bad,OUTPUT:comment,OUTPUT:is_good,GOLDEN:is_bad,GOLDEN:comment,GOLDEN:is_good,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,session_id,is_was_cool,is_was_not_cool,is_was_bad,is_was_not_bad
0,"{""action0"":{""answer"":""А мне нравятся оба"",""que...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2695e0d71d24f4cbc626,0000121611--628e2c725e80e7444c4f9ed9,ac9bd3ef62681c43f48d602dbaf32cee,APPROVED,2022-05-25T13:17:38.335,uu/2db448b76731726aa5f0dca6294ba245_1653019216...,0,1,1,0
1,"{""action0"":{""action"":""Включается видео после п...",no,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2693e0d71d24f4cbc603,0000121611--628e2db057f35d6abf7cafe7,ac9bd3ef62681c43f48d602dbaf32cee,APPROVED,2022-05-25T13:22:56.320,uu/c883736fd98eed55cec5ac5486521fa6_1653320019...,0,1,0,1
2,"{""action0"":{""action"":""Воспроизведение аудио ст...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2693e0d71d24f4cbc5ff,0000121611--628e2dc75e80e7444c4fa0c6,ac9bd3ef62681c43f48d602dbaf32cee,APPROVED,2022-05-25T13:23:19.204,uu/b76d48800d23dfe33364f731768672bc_1653149722...,0,1,1,0
3,"{""action0"":{""answer"":""Добрых снов. Зовите, ког...",no,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2695e0d71d24f4cbc624,0000121611--628e2e26e0d71d24f4cbd5ba,ac9bd3ef62681c43f48d602dbaf32cee,APPROVED,2022-05-25T13:24:54.384,uu/263b350270e6c25a73fc01124f000249_1653256491...,0,1,0,1
4,"{""action0"":{""action"":""Лампочка[1] меняет цвето...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2697e0d71d24f4cbc641,0000121611--628e2e6a5e80e7444c4fa184,ac9bd3ef62681c43f48d602dbaf32cee,APPROVED,2022-05-25T13:26:02.825,uu/6ba4c8c4b6469f2d7b87eee5c157c817_1653066484...,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,"{""action0"":{""action"":""Первый трек, который вкл...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2691e0d71d24f4cbc5dd,0000121611--62947126e0d71d24f4ce9fff,c69d857811ae0f7f3f6e3c5e569cfaad,APPROVED,2022-05-30T07:24:22.087,uu/9f068cdb5dc6d163640237ca514aaf2e_1653143400...,0,1,1,0
280,"{""action0"":{""answer"":""Всё в этом мире относите...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e268fe0d71d24f4cbc5b5,0000121611--6294717b5e80e7444c5368c4,c69d857811ae0f7f3f6e3c5e569cfaad,APPROVED,2022-05-30T07:25:47.837,uu/7d327c7706f1e31a41f71aab190ff46d_1652972406...,0,1,1,0
281,"{""action0"":{""answer"":""Везет вам! Тепло."",""quer...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2694e0d71d24f4cbc60d,0000121611--62947199e0d71d24f4cea021,c69d857811ae0f7f3f6e3c5e569cfaad,APPROVED,2022-05-30T07:26:17.121,uu/dc672c61b4854a544284ee95afd42d6d_1652939843...,0,1,1,0
282,"{""action0"":{""answer"":""Моя семья и друзья — это...",yes,NaN,no,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1185297...,0000121611--628e2694e0d71d24f4cbc610,0000121611--629471c65e80e7444c5368d8,c69d857811ae0f7f3f6e3c5e569cfaad,APPROVED,2022-05-30T07:27:02.019,uu/e4c480e4e8d6c7f3dad22b4158a573d0_1653238373...,0,1,1,0


In [7]:
aggregated_results = toloka_results.groupby(by=["session_id"]).agg(
                                            is_was_cool = pd.NamedAgg(column = 'is_was_cool', aggfunc = 'sum'),
                                            is_was_not_cool = pd.NamedAgg(column = 'is_was_not_cool', aggfunc = 'sum'),
                                            is_was_bad = pd.NamedAgg(column = 'is_was_bad', aggfunc = 'sum'),
                                            is_was_not_bad = pd.NamedAgg(column = 'is_was_not_bad', aggfunc = 'sum'),
                                            link = pd.NamedAgg(column = 'ASSIGNMENT:link', aggfunc = 'last'), 
                                            ).reset_index()

aggregated_results_by_worker = toloka_results.groupby(by=["ASSIGNMENT:worker_id"]).agg(
                                            is_was_cool = pd.NamedAgg(column = 'is_was_cool', aggfunc = 'sum'),
                                            is_was_not_cool = pd.NamedAgg(column = 'is_was_not_cool', aggfunc = 'sum'),
                                            is_was_bad = pd.NamedAgg(column = 'is_was_bad', aggfunc = 'sum'),
                                            is_was_not_bad = pd.NamedAgg(column = 'is_was_not_bad', aggfunc = 'sum'),
        
                                            ).reset_index()

In [8]:
aggregated_results_by_worker

,ASSIGNMENT:worker_id,is_was_cool,is_was_not_cool,is_was_bad,is_was_not_bad
0,62d3f41254aee9b8522e2453511ba80f,21,52,40,33
1,79733354c4f18f09d3863dae012a14c7,13,27,27,13
2,ac9bd3ef62681c43f48d602dbaf32cee,1,24,11,14
3,c11bafef9830c8fcf423d41bcda28b3b,11,62,46,27
4,c69d857811ae0f7f3f6e3c5e569cfaad,23,50,45,28


In [11]:
session_quality_workers_answers = []
session_quality_workers_cnt = []
session_quality_workers = []

for i, row in aggregated_results_by_worker.iterrows():
    for col in ["is_was_cool", "is_was_not_cool"]:
        session_quality_workers_answers.append(col)
        session_quality_workers.append(row["ASSIGNMENT:worker_id"])
        session_quality_workers_cnt.append(row[col])

df = pd.DataFrame({
    "worker": session_quality_workers,
    "answer": session_quality_workers_answers,
    "cnt": session_quality_workers_cnt,
})

import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df, x="worker", y="cnt", color="answer", title="Ответы размечающих")
fig.show()

session_quality_workers_answers = []
session_quality_workers_cnt = []
session_quality_workers = []

for i, row in aggregated_results_by_worker.iterrows():
    s = row[["is_was_cool", "is_was_not_cool"]].sum()
    
    
    for col in ["is_was_cool", "is_was_not_cool"]:
        session_quality_workers_answers.append(col)
        session_quality_workers.append(row["ASSIGNMENT:worker_id"])
        session_quality_workers_cnt.append(row[col]/s)

df = pd.DataFrame({
    "worker": session_quality_workers,
    "answer": session_quality_workers_answers,
    "cnt": session_quality_workers_cnt,
})

import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df, x="worker", y="cnt", color="answer", title="Доли ответов размечающих")
fig.show()

In [12]:
session_quality_workers_answers = []
session_quality_workers_cnt = []
session_quality_workers = []

for i, row in aggregated_results_by_worker.iterrows():
    for col in ["is_was_bad", "is_was_not_bad"]:
        session_quality_workers_answers.append(col)
        session_quality_workers.append(row["ASSIGNMENT:worker_id"])
        session_quality_workers_cnt.append(row[col])

df = pd.DataFrame({
    "worker": session_quality_workers,
    "answer": session_quality_workers_answers,
    "cnt": session_quality_workers_cnt,
})

import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df, x="worker", y="cnt", color="answer", title="Ответы размечающих")
fig.show()

session_quality_workers_answers = []
session_quality_workers_cnt = []
session_quality_workers = []

for i, row in aggregated_results_by_worker.iterrows():
    s = row[["is_was_bad", "is_was_not_bad"]].sum()
    
    
    for col in ["is_was_bad", "is_was_not_bad"]:
        session_quality_workers_answers.append(col)
        session_quality_workers.append(row["ASSIGNMENT:worker_id"])
        session_quality_workers_cnt.append(row[col]/s)

df = pd.DataFrame({
    "worker": session_quality_workers,
    "answer": session_quality_workers_answers,
    "cnt": session_quality_workers_cnt,
})

import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df, x="worker", y="cnt", color="answer", title="Доли ответов размечающих")
fig.show()

In [13]:
aggregated_results["session_id_old"] = aggregated_results["session_id"].apply(
    lambda x: "_".join(x.split("_")[:4])
)

In [21]:
ilnur_table = ilnur_table.rename(columns={"session_id": "session_id_old"})

In [23]:
test_id = ilnur_table.groupby(by="session_id_old").agg(
    test_name = pd.NamedAgg(column = 'test_name', aggfunc = 'last'),
    session_len = pd.NamedAgg(column = 'session_len', aggfunc = 'last'),
).reset_index()

In [24]:
aggregated_results = aggregated_results.merge(test_id, on="session_id_old")

In [150]:
aggregated_results

,session_id,is_was_cool,is_was_not_cool,is_was_bad,is_was_not_bad,link,session_id_old,test_name,session_len,is_was_cool_mv,is_was_bad_mv
0,uu/03f8714aa38db123d2c4a2ea2c51fe46_1653293668...,1,2,1,2,https://sandbox.toloka.yandex.com/task/1185297...,uu/03f8714aa38db123d2c4a2ea2c51fe46_1653293668...,cntrl_583841,12,0,0
1,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,3,1,1,3,https://sandbox.toloka.yandex.com/task/1185297...,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,test_583842,28,1,0
2,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,0,3,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,test_583842,28,0,1
3,uu/208d3abf370c7ff94bdb8e158c418413_1653076266...,2,1,3,0,https://sandbox.toloka.yandex.com/task/1185297...,uu/208d3abf370c7ff94bdb8e158c418413_1653076266...,test_583842,10,1,1
4,uu/263b350270e6c25a73fc01124f000249_1653247008...,0,4,2,2,https://sandbox.toloka.yandex.com/task/1185297...,uu/263b350270e6c25a73fc01124f000249_1653247008...,cntrl_583841,48,0,0
...,...,...,...,...,...,...,...,...,...,...,...
68,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,1,2,3,0,https://sandbox.toloka.yandex.com/task/1185297...,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,test_583842,21,0,1
69,uu/cc68e63c21406a957bec52c72c5fe031_1653262439...,0,3,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/cc68e63c21406a957bec52c72c5fe031_1653262439...,cntrl_583841,8,0,1
70,uu/dc672c61b4854a544284ee95afd42d6d_1652939843...,1,2,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/dc672c61b4854a544284ee95afd42d6d_1652939843...,cntrl_583841,9,0,1
71,uu/e4c480e4e8d6c7f3dad22b4158a573d0_1653238373...,0,3,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/e4c480e4e8d6c7f3dad22b4158a573d0_1653238373...,test_583842,9,0,1


### Согласованность

In [156]:
len(aggregated_results[
    (aggregated_results["is_was_cool"] == 0) |
    (aggregated_results["is_was_not_cool"] == 0)
])

34

In [153]:
len(aggregated_results[
    ((aggregated_results["is_was_cool"] < 2) & (aggregated_results["is_was_not_cool"] > 2)) |
    ((aggregated_results["is_was_not_cool"] < 2) & (aggregated_results["is_was_cool"] > 2))
])

50

In [154]:
len(aggregated_results[
    (aggregated_results["is_was_bad"] == 0) |
    (aggregated_results["is_was_not_bad"] == 0)
])

30

In [148]:
len(aggregated_results[
    ((aggregated_results["is_was_bad"] < 2) & (aggregated_results["is_was_not_bad"] > 2)) |
    ((aggregated_results["is_was_not_bad"] < 2) & (aggregated_results["is_was_bad"] > 2))
])

45

## Критерий для долей

In [36]:
aggregated_results["is_was_cool_mv"] = 1 * (aggregated_results["is_was_cool"] > aggregated_results["is_was_not_cool"])

aggregated_results["is_was_bad_mv"] = 1 * (aggregated_results["is_was_bad"] > aggregated_results["is_was_not_bad"])


In [37]:
aggregated_results

,session_id,is_was_cool,is_was_not_cool,is_was_bad,is_was_not_bad,link,session_id_old,test_name,session_len,is_was_cool_mv,is_was_bad_mv
0,uu/03f8714aa38db123d2c4a2ea2c51fe46_1653293668...,1,2,1,2,https://sandbox.toloka.yandex.com/task/1185297...,uu/03f8714aa38db123d2c4a2ea2c51fe46_1653293668...,cntrl_583841,12,0,0
1,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,3,1,1,3,https://sandbox.toloka.yandex.com/task/1185297...,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,test_583842,28,1,0
2,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,0,3,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/163a32f6f2808b700a6a5f8bc930d177_1653202318...,test_583842,28,0,1
3,uu/208d3abf370c7ff94bdb8e158c418413_1653076266...,2,1,3,0,https://sandbox.toloka.yandex.com/task/1185297...,uu/208d3abf370c7ff94bdb8e158c418413_1653076266...,test_583842,10,1,1
4,uu/263b350270e6c25a73fc01124f000249_1653247008...,0,4,2,2,https://sandbox.toloka.yandex.com/task/1185297...,uu/263b350270e6c25a73fc01124f000249_1653247008...,cntrl_583841,48,0,0
...,...,...,...,...,...,...,...,...,...,...,...
68,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,1,2,3,0,https://sandbox.toloka.yandex.com/task/1185297...,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,test_583842,21,0,1
69,uu/cc68e63c21406a957bec52c72c5fe031_1653262439...,0,3,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/cc68e63c21406a957bec52c72c5fe031_1653262439...,cntrl_583841,8,0,1
70,uu/dc672c61b4854a544284ee95afd42d6d_1652939843...,1,2,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/dc672c61b4854a544284ee95afd42d6d_1652939843...,cntrl_583841,9,0,1
71,uu/e4c480e4e8d6c7f3dad22b4158a573d0_1653238373...,0,3,2,1,https://sandbox.toloka.yandex.com/task/1185297...,uu/e4c480e4e8d6c7f3dad22b4158a573d0_1653238373...,test_583842,9,0,1


In [187]:
import math
from collections import Counter
import scipy.stats as st
import numpy as np

def ci_bounds(pos, n, confidence = 0.95):
    if n == 0:
        return [0, 1]
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * pos / n
    return [(phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n),
    (phat + z * z / (2 * n) + z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)
    ]

def norm_test(pos1, n1, pos2, n2):
    p1 = pos1 / n1
    p2 = pos2 / n2
    
    p = (pos1 + pos2) / (n1 + n2)
    
    s = np.sqrt(p * (1 - p) * (1 / n1 + 1 / n2) )
    
    z_stat = (p1 - p2)/s
    
    return 2*(1 - st.norm.cdf(abs(z_stat)))

In [188]:
aggregated_results.groupby(by=["test_name"]).agg(
                                            is_was_cool_mv = pd.NamedAgg(column = 'is_was_cool_mv', aggfunc = 'sum'),
                                            is_was_bad_mv = pd.NamedAgg(column = 'is_was_bad_mv', aggfunc = 'sum'),
                                            cnt = pd.NamedAgg(column = 'session_id', aggfunc = 'count'),
                                            ).reset_index()

,test_name,is_was_cool_mv,is_was_bad_mv,cnt
0,cntrl_583841,9,17,39
1,test_583842,5,23,34


In [189]:
print("Доля кайфуль")
print(f"Тест для долей good: {norm_test(9, 39, 5, 34)}")
print(f"Доверительные интервалы в контроле {ci_bounds(9, 39)}")
print(f"Доверительные интервалы в тесте {ci_bounds(5, 34)}")

Доля кайфуль
Тест для долей good: 0.3648315872574903
Доверительные интервалы в контроле [0.1264580412514285, 0.3833625722834177]
Доверительные интервалы в тесте [0.06449441444782457, 0.30128056495452155]


In [190]:
print("Доля провалов")
print(f"Тест для долей good: {norm_test(17, 39, 23, 34)}")
print(f"Доверительный интервал в контроле {ci_bounds(17, 39)}")
print(f"Доверительный интервал в тесте {ci_bounds(23, 34)}")

Доля провалов
Тест для долей good: 0.039386677202688514
Доверительный интервал в контроле [0.2930475387568963, 0.5902430835133053]
Доверительный интервал в тесте [0.508428945939082, 0.8086835643597449]


### David-skene

In [112]:
dawid_skene_good = read_result = pd.DataFrame(
    client.read_table("//home/voice/eliseevmax/tasks/VA-2793/cool_and_bad_dawid_skene_good", raw=False)
)

dawid_skene_bad = read_result = pd.DataFrame(
    client.read_table("//home/voice/eliseevmax/tasks/VA-2793/cool_and_bad_dawid_skene_bad", raw=False)
)

In [113]:
dawid_skene_good["session_id"] = dawid_skene_good["inputValues"].apply(lambda x: x["input"]["session_id"])
dawid_skene_good["session_id_old"] = dawid_skene_good["inputValues"].apply(lambda x: "_".join(
                            x["input"]["session_id"].split("_")[:4])                                                                
                                                )
dawid_skene_good["probability"] = dawid_skene_good["probability"].apply(lambda x: x['is_good'])

dawid_skene_bad["session_id"] = dawid_skene_bad["inputValues"].apply(lambda x: x["input"]["session_id"])
dawid_skene_bad["session_id_old"] = dawid_skene_bad["inputValues"].apply(lambda x: "_".join(
                            x["input"]["session_id"].split("_")[:4])                                                                
                                                )
dawid_skene_bad["probability"] = dawid_skene_bad["probability"].apply(lambda x: x['is_bad'])

In [114]:
dawid_skene_good["is_good"] = dawid_skene_good["outputValues"].apply(lambda x: 1*(x['is_good'] == "yes"))
dawid_skene_bad["is_bad"] = dawid_skene_bad["outputValues"].apply(lambda x: 1*(x['is_bad'] == "yes"))

In [115]:
dawid_skene_good = dawid_skene_good.drop(columns=["outputValues", "inputValues", "knownSolutions"])
dawid_skene_bad = dawid_skene_bad.drop(columns=["outputValues", "inputValues", "knownSolutions"])

In [116]:
dawid_skene = dawid_skene_good.merge(dawid_skene_bad, on=["session_id", "session_id_old"], suffixes=("_good", "_bad"))

In [117]:
dawid_skene = dawid_skene.merge(test_id, on="session_id_old")

In [118]:
dawid_skene

,probability_good,session_id,session_id_old,is_good,probability_bad,is_bad,test_name,session_len
0,0.992348,uu/c883736fd98eed55cec5ac5486521fa6_1653320019...,uu/c883736fd98eed55cec5ac5486521fa6_1653320019...,0,0.999764,0,test_583842,16
1,0.956572,uu/c883736fd98eed55cec5ac5486521fa6_1653320019...,uu/c883736fd98eed55cec5ac5486521fa6_1653320019...,0,0.994944,0,test_583842,16
2,0.544943,uu/f3de59f68952790990fec5e0297ad174_1653284281...,uu/f3de59f68952790990fec5e0297ad174_1653284281...,0,0.993224,1,test_583842,11
3,0.911870,uu/6ba4c8c4b6469f2d7b87eee5c157c817_1653296502...,uu/6ba4c8c4b6469f2d7b87eee5c157c817_1653296502...,0,0.999996,0,test_583842,36
4,0.912790,uu/6ba4c8c4b6469f2d7b87eee5c157c817_1653296502...,uu/6ba4c8c4b6469f2d7b87eee5c157c817_1653296502...,1,0.650821,1,test_583842,36
...,...,...,...,...,...,...,...,...
68,0.912790,uu/37f66ca43b7028a4c2553a071fe1ddc1_1653287829...,uu/37f66ca43b7028a4c2553a071fe1ddc1_1653287829...,1,0.901905,0,test_583842,10
69,0.590694,uu/7d327c7706f1e31a41f71aab190ff46d_1652972406...,uu/7d327c7706f1e31a41f71aab190ff46d_1652972406...,0,0.934001,0,cntrl_583841,5
70,0.992348,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,0,0.993224,1,test_583842,21
71,0.720686,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,uu/cbdf356201fdd3f6d97bad34495af92f_1653209730...,0,0.930237,1,test_583842,21


In [119]:
dawid_skene.groupby(by=["test_name"]).agg(
                                            is_good = pd.NamedAgg(column = 'is_good', aggfunc = 'sum'),
                                            is_bad = pd.NamedAgg(column = 'is_bad', aggfunc = 'sum'),
                                            cnt = pd.NamedAgg(column = 'session_id', aggfunc = 'count'),
                                            ).reset_index()

,test_name,is_good,is_bad,cnt
0,cntrl_583841,11,14,39
1,test_583842,8,17,34


In [191]:
print("Доля кайфуль")
print(f"Тест для долей good: {norm_test(11, 39, 8, 34)}")
print(f"Доверительные интервалы в контроле {ci_bounds(11, 39)}")
print(f"Доверительные интервалы в тесте {ci_bounds(8, 34)}")

Доля кайфуль
Тест для долей good: 0.6497145769615482
Доверительные интервалы в контроле [0.16543492088847678, 0.43775319483020825]
Доверительные интервалы в тесте [0.12436985532639575, 0.39996137922536384]


In [192]:
print("Доля провалов")
print(f"Тест для долей good: {norm_test(14, 39, 17, 34)}")
print(f"Доверительный интервал в контроле {ci_bounds(14, 39)}")
print(f"Доверительный интервал в тесте {ci_bounds(17, 34)}")

Доля провалов
Тест для долей good: 0.2239932335483319
Доверительный интервал в контроле [0.22742085458992867, 0.5158185144045146]
Доверительный интервал в тесте [0.34069325645682497, 0.659306743543175]


In [123]:
good_workers = pd.read_csv("cool_good_workers", sep="\t", header=None)

In [139]:
import plotly.express as px


for i, row in good_workers.iterrows():
    fig = px.imshow(
        np.array([float(item) for item in row[2].split(" ")[:-1]]).reshape(2, 2),
    )
    fig.update_layout(title = row[0])
    fig.show()

In [140]:
bad_workers = pd.read_csv("cool_bad_workers", sep="\t", header=None)

In [143]:
import plotly.express as px


for i, row in bad_workers.iterrows():
    fig = px.imshow(
        np.array([float(item) for item in row[2].split(" ")[:-1]]).reshape(2, 2),
    )
    fig.update_layout(title = row[0])
    fig.show()

### Метрика с кайфулями и провалами

In [185]:
def test_coefs(aggregated_results, bad_coef=-1, good_coef=1, test_name="test_583842", cntrl_name="cntrl_583841", return_values = False):
    aggregated_results["joint_metric"] = aggregated_results["is_was_cool_mv"] * good_coef + bad_coef * aggregated_results["is_was_bad_mv"]

    test_raw = aggregated_results[aggregated_results["test_name"] == test_name]["joint_metric"].values
    ctrl_raw = aggregated_results[aggregated_results["test_name"] == cntrl_name]["joint_metric"].values
    
    pvalue=mannwhitneyu(ctrl_raw, test_raw).pvalue
    effect = (test_raw.mean() - ctrl_raw.mean())/(ctrl_raw.mean() + 1e-8)
    
    if return_values:
        return pvalue, effect, ctrl_raw.mean(), test_raw.mean()
    return pvalue, effect

In [186]:
test_coefs(aggregated_results, return_values=True)

(0.02963272517394004,
 1.580882430009195,
 -0.20512820512820512,
 -0.5294117647058824)

In [181]:
grid_bad = np.linspace(-1, 0, 10)
grid_good = np.linspace(0, 1, 10)

good_coefs = []
bad_coefs = []

p_values = []
effects = []

for bad_coef in tqdm(grid_bad):
    
    loc_p_values = []
    loc_effects = []
    
    for good_coef in grid_good:
        
        p_value, effect = test_coefs(aggregated_results, good_coef=good_coef, bad_coef=bad_coef)
        
        loc_p_values.append(p_value)
        loc_effects.append(effect)
    
    p_values.append(loc_p_values)
    effects.append(loc_effects)


  0%|          | 0/10 [00:00<?, ?it/s]

In [182]:
fig = px.imshow(
    p_values,
    x = grid_good,
    y = grid_bad
)
fig.update_layout(title = "p_value")
fig.show()

In [183]:
fig = px.imshow(
    np.abs(effects),
    x = grid_good,
    y = grid_bad
)
fig.update_layout(title = "p_value")
fig.show()